In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate


In [9]:


chat = ChatOpenAI(
    temperature = 0.1, 
)


In [ ]:


template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)

prompt = template.format(country_a="SouthKorea", country_b="Japan")

chat.predict(prompt)

In [23]:
chat.predict(prompt)

'The distance between South Korea and Japan varies depending on the specific locations being compared. However, the shortest distance between the two countries is approximately 215 kilometers (134 miles) across the Korea Strait.'

In [27]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "안녕, 내 이름은 {name}!"),
        ("human", "What is the distance between {country_a} and {country_b}.\
                Also, what is your name?"),
    ]
)

In [32]:
prompt = template.format_messages(
    language = "Korean",
    name = "노석현",
    country_a = "Mexico",
    country_b = "Korea",
)

In [33]:
chat.predict_messages(prompt)

AIMessage(content='멕시코와 한국 사이의 거리는 대략 13,000km입니다. 제 이름은 노석현입니다.', response_metadata={'token_usage': <OpenAIObject at 0x7f6527ef05e0> JSON: {
  "prompt_tokens": 61,
  "completion_tokens": 38,
  "total_tokens": 99
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-01f3d375-7cf7-43e6-9a9b-0b2eb33836f3-0')

In [3]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items =  text.strip().split(",")
        return list(map(str.strip, items))

In [4]:
p  = CommaOutputParser()

p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [10]:
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

In [9]:
template = ChatPromptTemplate.from_messages([
    (
        "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else."
     ),
     ("human","{question}"),
    ]
)

prompt = template.format_messages(
    max_items = 10,
    question = "what are the colors?",
)

chat.predict_messages(prompt)

AIMessage(content='red, blue, green, yellow, orange, purple, pink, black, white, brown', response_metadata={'token_usage': <OpenAIObject at 0x7f84f95e9a80> JSON: {
  "prompt_tokens": 47,
  "completion_tokens": 19,
  "total_tokens": 66
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-065f305e-d39a-4bff-811a-8f2fd3066402-0')

In [1]:
from langchain.schema import BaseOutputParser

In [2]:
class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

In [4]:
from langchain.prompts import ChatPromptTemplate

In [11]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be \
            answered with a comma seperated list of max {max_items}. Do NOT reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)
prompt = template.format_messages(max_items = 10, question = "What are the planets?")

chat.predict_messages(prompt)



AIMessage(content='Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune, Pluto', response_metadata={'token_usage': <OpenAIObject at 0x7f007e4281d0> JSON: {
  "prompt_tokens": 49,
  "completion_tokens": 19,
  "total_tokens": 68
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a2fa441e-f1f7-4025-85da-c2f373b9f1ae-0')

In [12]:
result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['Mercury',
 'Venus',
 'Earth',
 'Mars',
 'Jupiter',
 'Saturn',
 'Uranus',
 'Neptune',
 'Pluto']

In [15]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 12,
    "question": "포켓몬은 무엇이니?"
})

['피카츄', '꼬부기', '파이리', '이상해씨', '꼬마돌', '피죤투', '꼬마돌', '파라스', '야도란', '디그다', '닥트리오']

## chain의 작동원리

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

In [23]:
# streaming은 응답이 보이게끔 설정
chat = ChatOpenAI(temperature = 0.1, 
                  streaming = True,
                  callbacks = [StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow \
     recipies for any type of cuisine with easy to find ingredients."),
     ("human", "I want to cook {cuisine} food"),
])

chef_chain = chef_prompt | chat

In [24]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies \
     vegetarian. You find alternative ingredients and explain their preparation. You \
     don't radically modify the recipe. If there is no alternative for a food just \
     say you don't know how to replace it."),
     ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat


In [25]:
final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "indian"
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic recipe for Chicken Tikka Masala, a popular dish enjoyed around the world.

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tbsp lemon juice
- 2 tsp ground cumin
- 2 tsp ground coriander
- 1 tsp turmeric
- 1 tsp garam masala
- 1 tsp paprika
- 1/2 tsp cayenne pepper (adjust to taste)
- Salt and pepper to taste
- 2 tbsp vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) crushed tomatoes
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, combine yogurt, lemon juice, cumin, coriander, turmeric, garam masala, paprika, cayenne pepper, salt, and pepper. Add the chicken pieces and marinate for at least 1 hour, or overnight for best results.

2. Preheat the oven to 400°F (200°C). Thread the marinated chicken onto skewers and place 

AIMessage(content="To make a vegetarian version of Chicken Tikka Masala, you can replace the chicken with a plant-based alternative such as tofu or paneer. Here's how you can adapt the recipe:\n\nIngredients:\n- 1 lb firm tofu or paneer, cut into bite-sized pieces\n- 1 cup plain yogurt (you can use dairy-free yogurt as a substitute)\n- 2 tbsp lemon juice\n- 2 tsp ground cumin\n- 2 tsp ground coriander\n- 1 tsp turmeric\n- 1 tsp garam masala\n- 1 tsp paprika\n- 1/2 tsp cayenne pepper (adjust to taste)\n- Salt and pepper to taste\n- 2 tbsp vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1-inch piece of ginger, grated\n- 1 can (14 oz) crushed tomatoes\n- 1 cup coconut cream (as a substitute for heavy cream)\n- Fresh cilantro, chopped (for garnish)\n\nInstructions:\n1. Follow the same marinating process as the original recipe, but instead of chicken, marinate the tofu or paneer in the yogurt and spice mixture for at least 1 hour.\n\n2. Bake the marinated tofu or pane